In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prana\\OneDrive\\Desktop\\Chicken-Disease-classificn\\research'

In [3]:
os.chdir("../") #go back one folder

In [4]:
%pwd

'c:\\Users\\prana\\OneDrive\\Desktop\\Chicken-Disease-classificn'

Entity is the written type of a function!!!

In [5]:
from dataclasses import dataclass

In [6]:
from pathlib import Path

@dataclass(frozen = True)  # are we creating a datatype here?
class DataIngestionCongig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from CNN_Classifier.constants import *
from CNN_Classifier.utilities.common import read_yaml, create_directories

Defining configuration manager

In [18]:
class ConfigurationManager:
    def __init__(
        self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path) # returns all the files from the yaml file
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root]) # since we used box type we can call the value using the "."
        
    def get_data_ingestion_config(self) -> DataIngestionCongig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionCongig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from CNN_Classifier import logger
from CNN_Classifier.utilities.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionCongig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  # get size give all the data of the file
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Data ingestion pipeline:!!!!!

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-29 15:44:52,362; INFO; common; yaml file: config\config.yaml loaded successfully]
[2023-12-29 15:44:52,366; INFO; common; yaml file: parameters.yaml loaded successfully]
[2023-12-29 15:44:52,370; INFO; common; created directory at: artifacts]
[2023-12-29 15:44:52,371; INFO; common; created directory at: artifacts/data_ingestion]
[2023-12-29 15:44:54,975; INFO; 1032179669; artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11621297
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9c59eb086f697a4734c2bda47f04f88206df50dfc5d0dcc683113e225dfde848"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E3D2:8283:4E9058:60F4AF:658F3DCF
Accept-Ranges: bytes
Date: Fri, 29 Dec 2023 21:44:48 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdal2120091-DFW
